In [10]:
import pandas as pd

m_df = pd.read_excel('./data/mercado-desafio.xlsx')
t_df = pd.read_excel('./data/transacoes-desafio.xlsx')

In [6]:
m_df.shape, t_df.shape

((308469, 9), (9513, 10))

In [11]:
m_df.head()

,Date,Company,Origin_city,Origin_state,Destination_city,Destination_state,Product,Price,CBOT
0,2024-01-30,Polaris,Abelardo Luz,SC,Joaçaba,SC,Soja,114.231354,1260.025702
1,2024-01-30,Polaris,Alegrete,RS,Rio Grande,RS,Soja,118.031576,1241.320557
2,2024-01-30,Polaris,Alta Floresta,MT,Barcarena,PA,Milho,31.075042,501.491344
3,2024-01-30,Polaris,Alta Floresta,MT,Barcarena,PA,Soja,94.684088,1173.122729
4,2024-01-30,Polaris,Alta Floresta,MT,Santos,SP,Milho,23.563284,419.767221


In [12]:
t_df.head()

,Date,Time,Company,Seller ID,Buyer ID,Price,Amount,Product,origin_city,origin_state
0,2024-02-28,11:44:47,Polaris,100000001,200000001,99.045785,569.703694,Soja,Uberlândia,MG
1,2024-06-13,14:01:54,Polaris,100000002,200000002,132.086256,4859.382786,Soja,Rondonópolis,MT
2,2024-06-13,14:12:18,Polaris,100000003,200000002,135.136955,15571.087533,Soja,Rondonópolis,MT
3,2024-06-26,14:27:55,Polaris,100000004,200000003,138.452293,10029.322768,Soja,Itiquira,MT
4,2024-06-26,14:30:48,Polaris,100000003,200000002,145.808870,19155.138463,Soja,Rondonópolis,MT


In [15]:
m_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308469 entries, 0 to 308468
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Date               308467 non-null  datetime64[ns]
 1   Company            308469 non-null  object        
 2   Origin_city        308466 non-null  object        
 3   Origin_state       308467 non-null  object        
 4   Destination_city   308465 non-null  object        
 5   Destination_state  308468 non-null  object        
 6   Product            308469 non-null  object        
 7   Price              308464 non-null  float64       
 8   CBOT               308466 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 21.2+ MB


In [16]:
t_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9513 entries, 0 to 9512
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          9513 non-null   datetime64[ns]
 1   Time          9513 non-null   object        
 2   Company       9513 non-null   object        
 3   Seller ID     9513 non-null   int64         
 4   Buyer ID      9513 non-null   int64         
 5   Price         9513 non-null   float64       
 6   Amount        9512 non-null   float64       
 7   Product       9513 non-null   object        
 8   origin_city   9511 non-null   object        
 9   origin_state  9513 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 743.3+ KB


In [21]:
m_df = m_df.drop(columns="Company", inplace=False, axis=1)

,Date,Origin_city,Origin_state,Destination_city,Destination_state,Product,Price,CBOT
0,2024-01-30,Abelardo Luz,SC,Joaçaba,SC,Soja,114.231354,1260.025702
1,2024-01-30,Alegrete,RS,Rio Grande,RS,Soja,118.031576,1241.320557
2,2024-01-30,Alta Floresta,MT,Barcarena,PA,Milho,31.075042,501.491344
3,2024-01-30,Alta Floresta,MT,Barcarena,PA,Soja,94.684088,1173.122729
4,2024-01-30,Alta Floresta,MT,Santos,SP,Milho,23.563284,419.767221


In [22]:
m_df.head()

,Date,Origin_city,Origin_state,Destination_city,Destination_state,Product,Price,CBOT
0,2024-01-30,Abelardo Luz,SC,Joaçaba,SC,Soja,114.231354,1260.025702
1,2024-01-30,Alegrete,RS,Rio Grande,RS,Soja,118.031576,1241.320557
2,2024-01-30,Alta Floresta,MT,Barcarena,PA,Milho,31.075042,501.491344
3,2024-01-30,Alta Floresta,MT,Barcarena,PA,Soja,94.684088,1173.122729
4,2024-01-30,Alta Floresta,MT,Santos,SP,Milho,23.563284,419.767221


In [26]:
t_df = t_df.drop(columns=['Company', 'Time'])

,Date,Seller ID,Buyer ID,Price,Amount,Product,origin_city,origin_state
0,2024-02-28,100000001,200000001,99.045785,569.703694,Soja,Uberlândia,MG
1,2024-06-13,100000002,200000002,132.086256,4859.382786,Soja,Rondonópolis,MT
2,2024-06-13,100000003,200000002,135.136955,15571.087533,Soja,Rondonópolis,MT
3,2024-06-26,100000004,200000003,138.452293,10029.322768,Soja,Itiquira,MT
4,2024-06-26,100000003,200000002,145.808870,19155.138463,Soja,Rondonópolis,MT


In [27]:
t_df.head()

,Date,Seller ID,Buyer ID,Price,Amount,Product,origin_city,origin_state
0,2024-02-28,100000001,200000001,99.045785,569.703694,Soja,Uberlândia,MG
1,2024-06-13,100000002,200000002,132.086256,4859.382786,Soja,Rondonópolis,MT
2,2024-06-13,100000003,200000002,135.136955,15571.087533,Soja,Rondonópolis,MT
3,2024-06-26,100000004,200000003,138.452293,10029.322768,Soja,Itiquira,MT
4,2024-06-26,100000003,200000002,145.808870,19155.138463,Soja,Rondonópolis,MT
